In [2]:
import ast
import json
import os
import torch

from tensordict.nn import TensorDictModule
from torchrl.collectors import SyncDataCollector
from torch.distributions import Categorical
from torchrl.envs.libs.pettingzoo import PettingZooWrapper
from torchrl.envs.transforms import TransformedEnv, RewardSum
from torchrl.envs.utils import check_env_specs
from torchrl.data.replay_buffers import ReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage
from torchrl.modules import MultiAgentMLP, ProbabilisticActor
from torchrl.objectives.value import GAE
from torchrl.objectives import ClipPPOLoss, ValueEstimators

from routerl import TrafficEnvironment
from tqdm import tqdm

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

/Users/akman/Desktop/URB/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
device = (
    torch.device(0)
    if torch.cuda.is_available()
    else torch.device("cpu")
)
print("device is: ", device)

device is:  cpu


#### Hyperparameters setting

In [6]:
params = json.load(open("../experiment_metadata.json"))
params = params["1"]["config"]
print(params)

{'exp_id': 1, 'seed': 42, 'new_machines_after_mutation': 50, 'frames_per_batch': 200, 'n_iters': 20, 'human_learning_episodes': 100, 'test_eps': 50, 'num_epochs': 1, 'minibatch_size': 16, 'lr': 0.0003, 'max_grad_norm': 3.0, 'clip_epsilon': 0.2, 'gamma': 0.99, 'lmbda': 0.9, 'entropy_eps': 0.0001, 'normalize_advantage': False, 'policy_network_depth': 3, 'policy_network_num_cells': 64, 'critic_network_depth': 3, 'critic_network_num_cells': 64, 'network': 'saint_arnoult', 'av_behavior': 'cooperative', 'smooth_by': 10, 'plot_choices': 'basic', 'human_model': 'gawron', 'noise_weight_agent': 0, 'noise_weight_path': 0.8, 'noise_weight_day': 0.2, 'beta': -1, 'beta_k_i_variability': 0.1, 'epsilon_i_variability': 0.1, 'epsilon_k_i_variability': 0.1, 'epsilon_k_i_t_variability': 0.1, 'greedy': 0.1, 'gamma_c': 0.0, 'gamma_u': 0.0, 'remember': 1, 'alpha_zero': 0.8, 'alphas': [0.2], 'number_of_paths': 4, 'path_gen_beta': -5, 'num_samples': 10}


In [7]:
# set params as variables in this notebook
for key, value in params.items():
    globals()[key] = value

In [ ]:
custom_network_folder = f"../networks/{network}"
training_episodes = (frames_per_batch / new_machines_after_mutation) * n_iters
total_frames = frames_per_batch * n_iters
phases = [0, human_learning_episodes, int(training_episodes) + human_learning_episodes]
phase_names = ["Human learning", "Mutation - Machine learning", "Testing phase"]
records_folder = f"records/{exp_id}"
plots_folder = f"plots/{exp_id}"

# Read origin-destinations
od_file_path = os.path.join(custom_network_folder, f"od_{network}.txt")
with open(od_file_path, 'r', encoding='utf-8') as f:
    content = f.read()
data = ast.literal_eval(content)
origins = data['origins']
destinations = data['destinations']

In [ ]:
# Copy agents.csv from custom_network_folder to records_folder
agents_csv_path = os.path.join(custom_network_folder, "agents.csv")
if os.path.exists(agents_csv_path):
    os.makedirs(records_folder, exist_ok=True)
    new_agents_csv_path = os.path.join(records_folder, "agents.csv")
    with open(agents_csv_path, 'r', encoding='utf-8') as f:
        content = f.read()
    with open(new_agents_csv_path, 'w', encoding='utf-8') as f:
        f.write(content)

In [ ]:
env = TrafficEnvironment(
    seed = seed,
    create_agents = False,
    create_paths = True,
    save_detectors_info = False,
    agent_parameters = {
        "new_machines_after_mutation": new_machines_after_mutation, 
        "human_parameters" : {
            "model" : human_model, 
            "noise_weight_agent" : noise_weight_agent,
            "noise_weight_path" : noise_weight_path,
            "noise_weight_day" : noise_weight_day,
            "beta" : beta,
            "beta_k_i_variability" : beta_k_i_variability,
            "epsilon_i_variability" : epsilon_i_variability,
            "epsilon_k_i_variability" : epsilon_k_i_variability,
            "epsilon_k_i_t_variability" : epsilon_k_i_t_variability,
            "greedy" : greedy,
            "gamma_c" : gamma_c,
            "gamma_u" : gamma_u,
            "remember" : remember,
            "alpha_zero" : alpha_zero,
            "alphas" : alphas
            },
        "machine_parameters" :{
            "behavior" : av_behavior,
            }
        },
    simulator_parameters = {
        "network_name" : network,
        "custom_network_folder" : custom_network_folder,
        "sumo_type" : "sumo",
    }, 
    plotter_parameters = {
        "phases" : phases,
        "phase_names" : phase_names,
        "smooth_by" : smooth_by,
        "plot_choices" : plot_choices,
        "records_folder" : records_folder,
        "plots_folder" : plots_folder
    },
    path_generation_parameters = {
        "origins" : origins,
        "destinations" : destinations,
        "number_of_paths" : number_of_paths,
        "beta" : path_gen_beta,
        "num_samples" : num_samples,
        "visualize_paths" : False
    } 
)

In [ ]:
print("Number of total agents is: ", len(env.all_agents), "\n")
print("Number of human agents is: ", len(env.human_agents), "\n")
print("Number of machine agents (autonomous vehicles) is: ", len(env.machine_agents), "\n")

In [ ]:
env.start()
res = env.reset()

#### Human learning

In [ ]:
for episode in range(human_learning_episodes):
    env.step()

#### Mutation

In [ ]:
env.mutation()

In [ ]:
print("Number of total agents is: ", len(env.all_agents), "\n")
print("Number of human agents is: ", len(env.human_agents), "\n")
print("Number of machine agents (autonomous vehicles) is: ", len(env.machine_agents), "\n")

In [ ]:
group = {'agents': [str(machine.id) for machine in env.machine_agents]}

env = PettingZooWrapper(
    env=env,
    use_mask=True,
    categorical_actions=True,
    done_on_any = False,
    group_map=group,
    device=device
)

In [ ]:
env = TransformedEnv(
    env,
    RewardSum(in_keys=[env.reward_key], out_keys=[("agents", "episode_reward")]),
)

In [ ]:
check_env_specs(env)


In [ ]:
reset_td = env.reset()

In [ ]:
share_parameters_policy = False 

policy_net = torch.nn.Sequential(
    MultiAgentMLP(
        n_agent_inputs = env.observation_spec["agents", "observation"].shape[-1],
        n_agent_outputs = env.action_spec.space.n,
        n_agents = env.n_agents,
        centralised=False,
        share_params=share_parameters_policy,
        device=device,
        depth=policy_network_depth,
        num_cells=policy_network_num_cells,
        activation_class=torch.nn.Tanh,
    ),
)

In [ ]:
policy_module = TensorDictModule(
    policy_net,
    in_keys=[("agents", "observation")],
    out_keys=[("agents", "logits")],
) 

In [ ]:
policy = ProbabilisticActor(
    module=policy_module,
    spec=env.action_spec,
    in_keys=[("agents", "logits")],
    out_keys=[env.action_key],
    distribution_class=Categorical,
    return_log_prob=True,
    log_prob_key=("agents", "sample_log_prob"),
)

In [ ]:
share_parameters_critic = True
mappo = False  # IPPO if False

critic_net = MultiAgentMLP(
    n_agent_inputs=env.observation_spec["agents", "observation"].shape[-1],
    n_agent_outputs=1, 
    n_agents=env.n_agents,
    centralised=mappo,
    share_params=share_parameters_critic,
    device=device,
    depth=critic_network_depth,
    num_cells=critic_network_num_cells,
    activation_class=torch.nn.ReLU,
)

critic = TensorDictModule(
    module=critic_net,
    in_keys=[("agents", "observation")],
    out_keys=[("agents", "state_value")],
)

#### Collector

In [ ]:
collector = SyncDataCollector(
    env,
    policy,
    device=device,
    storing_device=device,
    frames_per_batch=frames_per_batch,
    total_frames=total_frames,
) 

#### Replay buffer

In [ ]:
replay_buffer = ReplayBuffer(
    storage=LazyTensorStorage(
        frames_per_batch, device=device
    ),  
    sampler=SamplerWithoutReplacement(),
    batch_size=minibatch_size,
)

#### PPO loss function

In [ ]:
loss_module = ClipPPOLoss(
    actor_network=policy,
    critic_network=critic,
    clip_epsilon=clip_epsilon,
    entropy_coef=entropy_eps,
    normalize_advantage=normalize_advantage,
)
loss_module.set_keys( 
    reward=env.reward_key,  
    action=env.action_key, 
    sample_log_prob=("agents", "sample_log_prob"),
    value=("agents", "state_value"),
    done=("agents", "done"),
    terminated=("agents", "terminated"),
)

loss_module.make_value_estimator(
    ValueEstimators.GAE, gamma=gamma, lmbda=lmbda
) 

GAE = loss_module.value_estimator

optim = torch.optim.Adam(loss_module.parameters(), lr)

#### Training loop

In [ ]:
pbar = tqdm(total=n_iters, desc="episode_reward_mean = 0")

episode_reward_mean_list = []
loss_values = []
loss_entropy = []
loss_objective = []
loss_critic = []

for tensordict_data in collector: ##loops over frame_per_batch

    ## Generate the rollouts
    tensordict_data.set(
        ("next", "agents", "done"),
        tensordict_data.get(("next", "done"))
        .unsqueeze(-1)
        .expand(tensordict_data.get_item_shape(("next", env.reward_key))),  # Adjust index to start from 0
    )
    tensordict_data.set(
        ("next", "agents", "terminated"),
        tensordict_data.get(("next", "terminated"))
        .unsqueeze(-1)
        .expand(tensordict_data.get_item_shape(("next", env.reward_key))),  # Adjust index to start from 0
    )

    # Compute GAE for all agents
    with torch.no_grad():
            GAE(
                tensordict_data,
                params=loss_module.critic_network_params,
                target_params=loss_module.target_critic_network_params,
            )

    data_view = tensordict_data.reshape(-1)  
    replay_buffer.extend(data_view)

    ## Update the policies of the learning agents
    for _ in range(num_epochs):
        for _ in range(frames_per_batch // minibatch_size):
            subdata = replay_buffer.sample()
            loss_vals = loss_module(subdata)

            loss_value = (
                loss_vals["loss_objective"]
                + loss_vals["loss_critic"]
                + loss_vals["loss_entropy"]
            )

            loss_value.backward()

            torch.nn.utils.clip_grad_norm_(
                loss_module.parameters(), max_grad_norm
            ) 

            optim.step()
            optim.zero_grad()

            loss_values.append(loss_value.item())

            loss_entropy.append(loss_vals["loss_entropy"].item())

            loss_objective.append(loss_vals["loss_objective"].item())

            loss_critic.append(loss_vals["loss_critic"].item())


   
    collector.update_policy_weights_()
   
    # Logging
    done = tensordict_data.get(("next", "agents", "done"))  # Get done status for the group

    episode_reward_mean = (
        tensordict_data.get(("next", "agents", "episode_reward"))[done].mean().item()
    )
    episode_reward_mean_list.append(episode_reward_mean)


    pbar.set_description(f"episode_reward_mean = {episode_reward_mean}", refresh=False)
    pbar.update()

In [ ]:
policy.eval() # set the policy into evaluation mode
for episode in range(test_eps):
    env.rollout(len(env.machine_agents), policy=policy)

In [ ]:
env.plot_results()

In [ ]:
env.stop_simulation()